In [2]:
# Check cuda visible devices
import torch

print(torch.cuda.is_available())

print(torch.cuda.device_count())

True
2


In [3]:
%load_ext autoreload
%autoreload 2
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "CohereForAI/aya-101"
tokenizer = AutoTokenizer.from_pretrained(model_id)

aya_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    device_map="auto"
)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

model-00001-of-00011.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00011.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

model-00009-of-00011.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00010-of-00011.safetensors:   0%|          | 0.00/2.99G [00:00<?, ?B/s]

model-00011-of-00011.safetensors:   0%|          | 0.00/4.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [26]:
from IPython.display import display, clear_output, HTML
import textwrap

prompt = """Traducir del español ibérico al español rioplatense.

###
español: Hola hombre, ¿cómo estás?
rioplatense:¿Qué onda, chabón? ¿Cómo va?
###
Traducir del español al rioplatense

español: Vaya tío, eres un cabrón
rioplatense: Che pedazo de pelotudo, sos un forro
###
Traducir del español al rioplatense

español: ¡Tú puedes hacerlo!
"""
inputs = tokenizer(prompt, return_tensors="pt")
inputs = {k:v.to(aya_model.device) for k,v in inputs.items()}

output = aya_model.generate(
    **inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id, do_sample=True,
    top_p=0.95,
)

text = tokenizer.decode(output[0], skip_special_tokens=True)

clear_output(wait=True)
display(HTML(text))


In [ ]:
from datasets import load_dataset

dataset_name = "piuba-bigdata/contextualized_hate_speech"

ds = load_dataset(dataset_name)

In [ ]:
test_ds = ds["test"].shuffle()

In [ ]:
from pysentimiento.preprocessing import preprocess_tweet
import re

url_regex = r"\burl\b"


def text_preprocess(text):
    text = preprocess_tweet(text, preprocess_hashtags=False, demoji=False)
    text = text.replace("@usuario", "")

    text = re.sub(url_regex, "", text)
    # Replace multiple spaces with one
    text = " ".join(text.split())
    text = text.replace("\n", " ")

    return text


In [ ]:
tokenizer.model_max_length = 6400
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
from rioplatense_hs.mixtral import get_prompt

def tokenize(example):
    texto = text_preprocess(example["text"])

    contexto = preprocess_tweet(
        example["context_tweet"],
        preprocess_hashtags=False,
        demoji=False,
        preprocess_handles=False,
    )
    model_input = get_prompt(context=contexto, text=texto)
    return tokenizer(model_input, truncation=True)

tokenized_ds = test_ds.map(tokenize, batched=False)

In [ ]:
# prompt: Sort tokenized_ds by length of input_ids
tokenized_ds = tokenized_ds.map(lambda x: {"len": len(x["input_ids"])}, batched=False)

sorted_tokenized_ds = tokenized_ds.sort("len")

In [ ]:
# decode first example

inputs = sorted_tokenized_ds[0]["input_ids"]

text = tokenizer.decode(inputs, skip_special_tokens=True)

text = text.replace("[INST]", "\n[INST]")
text = text.replace("[/INST]", "[/INST]\n")

print(text)

In [ ]:
# prompt: Build a dataloader with a Causal LLM collator

from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
batch_size = 16
collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

def collate(inputs):
    attention = [ex["attention_mask"] for ex in inputs]
    input_ids = [ex["input_ids"] for ex in inputs]
    ids = [ex["id"] for ex in inputs]
    return ids, tokenizer.pad({"input_ids": input_ids, "attention_mask": attention}, return_tensors="pt")


dataloader = DataLoader(
    sorted_tokenized_ds,
    batch_size=batch_size,
    collate_fn=collate,
    shuffle=False,
    pin_memory=True,
    num_workers=16,
)


In [ ]:
from tqdm.auto import tqdm


for ids, inputs in tqdm(dataloader):
    output = model.generate(**inputs, max_new_tokens=150)

    for k, id in enumerate(ids):
        output_text = tokenizer.decode(output[k], skip_special_tokens=True)

        #with open(path+"/{}.txt".format(id), "w") as f:
        #    f.write(output_text)

    #for _ in range(4):
    #    print("="*80)
    #print(f"contexto: {contexto}")
    #print(f"texto: {texto}")
    #print(output_text)
